In [1]:
import sys

In [2]:
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/svu/e0384946/.ipython']

In [3]:
sys.path.append("/home/svu/e0384946/anaconda3/lib/python3.7/site-packages")

In [4]:
%cd Strong_Baseline_of_Pedestrian_Attribute_Recognition

/home/svu/e0384946/Strong_Baseline_of_Pedestrian_Attribute_Recognition


In [5]:
import os
import pprint
from collections import OrderedDict, defaultdict

import numpy as np
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

from batch_engine import valid_trainer, batch_trainer
from config import argument_parser
from dataset.AttrDataset import AttrDataset, get_transform
from loss.CE_loss import CEL_Sigmoid
from models.base_block import FeatClassifier, BaseClassifier
from models.resnet import resnet50
from tools.function import get_model_log_path, get_pedestrian_metrics
from tools.utils import time_str, save_ckpt, ReDirectSTD, set_seed

set_seed(605)

In [6]:
def main(args):
    visenv_name = args.dataset
    exp_dir = os.path.join('exp_result', args.dataset)
    model_dir, log_dir = get_model_log_path(exp_dir, visenv_name)
    stdout_file = os.path.join(log_dir, f'stdout_{time_str()}.txt')
    save_model_path = os.path.join(model_dir, 'ckpt_max.pth')

    if args.redirector:
        print('redirector stdout')
        ReDirectSTD(stdout_file, 'stdout', False)

    pprint.pprint(OrderedDict(args.__dict__))

    print('-' * 60)
    print(f'use GPU{args.device} for training')
    print(f'train set: {args.dataset} {args.train_split}, test set: {args.valid_split}')

    train_tsfm, valid_tsfm = get_transform(args)
    print(train_tsfm)

    train_set = AttrDataset(args=args, split=args.train_split, transform=train_tsfm)

    train_loader = DataLoader(
        dataset=train_set,
        batch_size=args.batchsize,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )
    valid_set = AttrDataset(args=args, split=args.valid_split, transform=valid_tsfm)

    valid_loader = DataLoader(
        dataset=valid_set,
        batch_size=args.batchsize,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
    )

    print(f'{args.train_split} set: {len(train_loader.dataset)}, '
          f'{args.valid_split} set: {len(valid_loader.dataset)}, '
          f'attr_num : {train_set.attr_num}')

    labels = train_set.label
    sample_weight = labels.mean(0)

    backbone = resnet50()
    classifier = BaseClassifier(nattr=train_set.attr_num)
    model = FeatClassifier(backbone, classifier)

    if torch.cuda.is_available():
        model = torch.nn.DataParallel(model).cuda()

    criterion = CEL_Sigmoid(sample_weight)

    param_groups = [{'params': model.module.finetune_params(), 'lr': args.lr_ft},
                    {'params': model.module.fresh_params(), 'lr': args.lr_new}]
    optimizer = torch.optim.SGD(param_groups, momentum=args.momentum, weight_decay=args.weight_decay, nesterov=False)
    lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=4)

    best_metric, epoch = trainer(epoch=args.train_epoch,
                                 model=model,
                                 train_loader=train_loader,
                                 valid_loader=valid_loader,
                                 criterion=criterion,
                                 optimizer=optimizer,
                                 lr_scheduler=lr_scheduler,
                                 path=save_model_path)

    print(f'{visenv_name},  best_metrc : {best_metric} in epoch{epoch}')


def trainer(epoch, model, train_loader, valid_loader, criterion, optimizer, lr_scheduler,
            path):
    maximum = float(-np.inf)
    best_epoch = 0

    result_list = defaultdict()

    for i in range(epoch):

        train_loss, train_gt, train_probs = batch_trainer(
            epoch=i,
            model=model,
            train_loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
        )

        valid_loss, valid_gt, valid_probs = valid_trainer(
            model=model,
            valid_loader=valid_loader,
            criterion=criterion,
        )

        lr_scheduler.step(metrics=valid_loss, epoch=i)

        train_result = get_pedestrian_metrics(train_gt, train_probs)
        valid_result = get_pedestrian_metrics(valid_gt, valid_probs)

        print(f'Evaluation on test set, \n',
              'ma: {:.4f},  pos_recall: {:.4f} , neg_recall: {:.4f} \n'.format(
                  valid_result.ma, np.mean(valid_result.label_pos_recall), np.mean(valid_result.label_neg_recall)),
              'Acc: {:.4f}, Prec: {:.4f}, Rec: {:.4f}, F1: {:.4f}'.format(
                  valid_result.instance_acc, valid_result.instance_prec, valid_result.instance_recall,
                  valid_result.instance_f1))

        print(f'{time_str()}')
        print('-' * 60)

        cur_metric = valid_result.ma

        if cur_metric > maximum:
            maximum = cur_metric
            best_epoch = i
            save_ckpt(model, path, i, maximum)

        result_list[i] = [train_result, valid_result]

    torch.save(result_list, os.path.join(os.path.dirname(path), 'metric_log.pkl'))

    return maximum, best_epoch

In [7]:
class args_config():
    def __init__(self):
        self.dataset="RAP"
        self.debug='store_false'
        self.batchsize=64
        self.train_epoch=30
        self.height=256
        self.width=192
        self.lr_ft=0.01
        self.lr_new=0.1
        self.classifier='base'
        self.momentum=0.9
        self.weight_decay=5e-4
        self.train_split="trainval" #choices=['train' 'trainval']
        self.valid_split="test" #choices=['test' 'valid']
        self.device=0
        self.redirector='store_false'
        self.use_bn='store_false'

In [8]:
args = args_config()

In [10]:
args.dataset = "PETA"

In [11]:
main(args)

redirector stdout
OrderedDict([('dataset', 'PETA'),
             ('debug', 'store_false'),
             ('batchsize', 64),
             ('train_epoch', 30),
             ('height', 256),
             ('width', 192),
             ('lr_ft', 0.01),
             ('lr_new', 0.1),
             ('classifier', 'base'),
             ('momentum', 0.9),
             ('weight_decay', 0.0005),
             ('train_split', 'trainval'),
             ('valid_split', 'test'),
             ('device', 0),
             ('redirector', 'store_false'),
             ('use_bn', 'store_false')])
------------------------------------------------------------
use GPU0 for training
train set: PETA trainval, test set: test
Compose(
    Resize(size=(256, 192), interpolation=PIL.Image.BILINEAR)
    Pad(padding=10, fill=0, padding_mode=constant)
    RandomCrop(size=(256, 192), padding=None)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
trainval set:

  0%|          | 0/297 [00:00<?, ?it/s]

2020-06-07_12:20:56, Step 326/327 in Ep 0, 0.13s  train_loss:26.3594
Epoch 0, LR 0.1, Train_Time 82.08s, Loss: 35.0634


100%|██████████| 297/297 [01:34<00:00,  3.16it/s]


Evaluation on test set, 
 ma: 0.7305,  pos_recall: 0.5032 , neg_recall: 0.9578 
 Acc: 0.7093, Prec: 0.8265, Rec: 0.8058, F1: 0.8136
2020-06-07_12:22:30
------------------------------------------------------------
2020-06-07_12:22:37, Step 19/327 in Ep 1, 0.21s  train_loss:21.8404
2020-06-07_12:22:41, Step 39/327 in Ep 1, 0.21s  train_loss:18.2222
2020-06-07_12:22:45, Step 59/327 in Ep 1, 0.21s  train_loss:21.5531
2020-06-07_12:22:49, Step 79/327 in Ep 1, 0.21s  train_loss:21.8963
2020-06-07_12:22:53, Step 99/327 in Ep 1, 0.21s  train_loss:21.6304
2020-06-07_12:22:57, Step 119/327 in Ep 1, 0.21s  train_loss:22.0239
2020-06-07_12:23:02, Step 139/327 in Ep 1, 0.21s  train_loss:21.6277
2020-06-07_12:23:06, Step 159/327 in Ep 1, 0.21s  train_loss:22.7798
2020-06-07_12:23:10, Step 179/327 in Ep 1, 0.21s  train_loss:21.4556
2020-06-07_12:23:14, Step 199/327 in Ep 1, 0.21s  train_loss:19.2526
2020-06-07_12:23:18, Step 219/327 in Ep 1, 0.21s  train_loss:21.8597
2020-06-07_12:23:23, Step 239/327

...

Evaluation on test set, 
 ma: 0.9208,  pos_recall: 0.8525 , neg_recall: 0.9891 
 Acc: 0.9234, Prec: 0.9563, Rec: 0.9475, F1: 0.9513
2020-06-07_13:26:27
------------------------------------------------------------
2020-06-07_13:26:31, Step 19/327 in Ep 29, 0.20s  train_loss:1.0424
2020-06-07_13:26:35, Step 39/327 in Ep 29, 0.20s  train_loss:1.1418
2020-06-07_13:26:39, Step 59/327 in Ep 29, 0.20s  train_loss:1.0206
2020-06-07_13:26:43, Step 79/327 in Ep 29, 0.21s  train_loss:0.8926
2020-06-07_13:26:47, Step 99/327 in Ep 29, 0.20s  train_loss:0.9420
2020-06-07_13:26:52, Step 119/327 in Ep 29, 0.20s  train_loss:0.9483
2020-06-07_13:26:56, Step 139/327 in Ep 29, 0.20s  train_loss:1.3572
2020-06-07_13:27:00, Step 159/327 in Ep 29, 0.20s  train_loss:1.1028
2020-06-07_13:27:04, Step 179/327 in Ep 29, 0.20s  train_loss:1.1209
2020-06-07_13:27:08, Step 199/327 in Ep 29, 0.20s  train_loss:0.9966
2020-06-07_13:27:12, Step 219/327 in Ep 29, 0.21s  train_loss:0.9885
2020-06-07_13:27:16, Step 239/327

  0%|          | 0/297 [00:00<?, ?it/s]

2020-06-07_13:27:34, Step 326/327 in Ep 29, 0.13s  train_loss:1.6580
Epoch 29, LR 0.010000000000000002, Train_Time 67.83s, Loss: 1.0387


100%|██████████| 297/297 [01:07<00:00,  4.42it/s]


Evaluation on test set, 
 ma: 0.9221,  pos_recall: 0.8552 , neg_recall: 0.9891 
 Acc: 0.9234, Prec: 0.9560, Rec: 0.9478, F1: 0.9513
2020-06-07_13:28:42
------------------------------------------------------------
PETA,  best_metrc : 0.9221381206976946 in epoch29
